In [1]:
!pip install datasets
!pip install evaluate
!pip install accelerate -U
!pip install peft
!pip install --upgrade pyarrow
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.9 M

In [2]:
import torch
import evaluate
import numpy as np
import pandas as pd
from transformers import (AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig)
from datasets import (load_dataset, Dataset)
from torch.utils.data import DataLoader
from torch.optim import AdamW
from peft import LoraConfig, get_peft_model
from google.colab import drive

In [3]:
torch.manual_seed(42)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
import torch
import gc
import os

def clear_cache():
  for _ in range(5):
    gc.collect()
    torch.cuda.empty_cache()
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [8]:
data_dir = "/content/drive/MyDrive/"
# data_csv = data_dir + "Subset2.csv"

In [9]:
train_df = pd.read_csv(data_dir+ "train.csv")
validation_df = pd.read_csv(data_dir + "test.csv")

train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)

In [10]:
# prompt: do a loop that goes over both datasets that we created and prints the number of post with column label = 1 and column label = 0... print also the name (Train, validation and test) of the dataset

for dataset, name in zip([train_dataset, validation_dataset, ], ["Train", "Validation", ]):
  num_label_1 = len(dataset.filter(lambda example: example['label'] == 1))
  num_label_0 = len(dataset.filter(lambda example: example['label'] == 0))
  print(f"{name} Dataset:")
  print(f"Number of posts with label 1: {num_label_1}")
  print(f"Number of posts with label 0: {num_label_0}")
  print("----")


Filter:   0%|          | 0/36640 [00:00<?, ? examples/s]

Filter:   0%|          | 0/36640 [00:00<?, ? examples/s]

Train Dataset:
Number of posts with label 1: 18012
Number of posts with label 0: 18628
----


Filter:   0%|          | 0/12214 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12214 [00:00<?, ? examples/s]

Validation Dataset:
Number of posts with label 1: 5965
Number of posts with label 0: 6249
----


In [11]:
GEMMA = "google/gemma-2b"

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(
        example['body'],
        padding="max_length",
        truncation=True,
        max_length=512  # Adjust max_length based on your needs
    )

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = validation_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Map:   0%|          | 0/36640 [00:00<?, ? examples/s]

Map:   0%|          | 0/12214 [00:00<?, ? examples/s]

In [13]:
def format_labels(example):
    example['labels'] = example['label']
    return example

tokenized_train_dataset = tokenized_train_dataset.map(format_labels)
tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

tokenized_val_dataset = tokenized_val_dataset.map(format_labels)
tokenized_val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/36640 [00:00<?, ? examples/s]

Map:   0%|          | 0/12214 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoModelForSequenceClassification
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForSequenceClassification.from_pretrained(
    "google/gemma-2b",
    num_labels=2,
    quantization_config=quantization_config,
    device_map="auto",
    pad_token_id=tokenizer.pad_token_id,
    offload_folder="content/drive/MyDrive/offload", # Specify a folder to offload parts of the model to disk
)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of GemmaForSequenceClassification were not initialized from the model checkpoint at google/gemma-2b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    modules_to_save=["score"],
    task_type=TaskType.SEQ_CLS,
)

model = get_peft_model(model, lora_config)

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids
    preds = np.argmax(predictions, axis=1)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    accuracy = accuracy_score(labels, preds)
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall}

In [17]:
BATCH_SIZE = 64

In [18]:
clear_cache()

In [19]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
    save_total_limit=3,
    save_strategy="epoch",
    label_names=["labels"],
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two 

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall
1,0.421200,0.430405,0.785328,0.849467,0.681140
2,0.408700,0.404442,0.803177,0.876030,0.695390
3,0.340600,0.398790,0.807516,0.887602,0.693713


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/ch

TrainOutput(global_step=1719, training_loss=0.445181140516857, metrics={'train_runtime': 7191.1241, 'train_samples_per_second': 15.286, 'train_steps_per_second': 0.239, 'total_flos': 6.704788824863539e+17, 'train_loss': 0.445181140516857, 'epoch': 3.0})

In [21]:
# Save the fine-tuned model
trainer.save_model("/content/drive/MyDrive/gemma_2b_fine_tuned")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/gemma_2b_fine_tuned_tokenizer")

('/content/drive/MyDrive/gemma_2b_fine_tuned_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/gemma_2b_fine_tuned_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/gemma_2b_fine_tuned_tokenizer/tokenizer.model',
 '/content/drive/MyDrive/gemma_2b_fine_tuned_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/gemma_2b_fine_tuned_tokenizer/tokenizer.json')

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
